In [76]:
# Load model directly
from transformers import AutoTokenizer, Gemma2ForCausalLM

hf_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", use_fast=True)
tr_tokenizer = AutoTokenizer.from_pretrained("alibayram/tr_tokenizer", use_fast=True)
umut_tokenizer = AutoTokenizer.from_pretrained("umutckmk2/tr_tokenizer", use_fast=True)
arda_tokenizer = AutoTokenizer.from_pretrained("aliarda/turkish_tokenizer_256k", use_fast=True)


tokenizer_config.json:   0%|          | 0.00/4.34M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [ ]:
umut_tokenizer.get_vocab()

79078

In [39]:
hf_model = Gemma2ForCausalLM.from_pretrained("google/gemma-2-2b-it")
hf_model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps

In [3]:
example_text = "Nasılsın, iyi misin? Türkçe cevap ver lütfen!"
input_ids = hf_tokenizer.encode(example_text, return_tensors="pt")
input_ids

tensor([[     2, 235300,  25725,   1560,   4119, 235269,  42233,   2586,    473,
         235336, 102823, 153384,    972,    533, 173077, 235341]])

In [4]:
# before changing of the embedding layer
output_token_ids = hf_model.generate(input_ids, max_length=100)
output_token_ids, hf_tokenizer.decode(output_token_ids[0])

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


(tensor([[     2, 235300,  25725,   1560,   4119, 235269,  42233,   2586,    473,
          235336, 102823, 153384,    972,    533, 173077, 235341,    109,    688,
          178229, 235341,   4658,    571, 190404,  14630,  56292, 235265,    139,
           13625,   7863, 235405,   1560,   4119, 235336, 206506,   2586,    473,
          235336,    688,    109,    688,   7510,  42233,  89249, 235269, 162625,
          165296, 235265,   5620,   7863, 235405,   1560,   4119, 235336,    688,
             109,    688, 235070,  89249, 235269, 162625, 165296, 235265,    139,
           13625,    679,  18168, 182564,    549, 235336,    688,    109,    688,
            7510,    581,   2843,  15128, 122469,  89249, 235269, 102823,  80781,
           21032,  44875, 235265,    139,  13625,    679,  18168, 182564,    549,
          235336,    688,    109,    688,   7510,    581, 102823,  80781,  21032,
           44875]]),
 '<bos>Nasılsın, iyi misin? Türkçe cevap ver lütfen!\n\n**Merhaba! Benim adım

In [5]:
hf_model.lm_head.weight.dtype

torch.float32

In [ ]:
# torch.FloatTensor

In [6]:
import sys
from multiprocessing import Pool
from multiprocessing.reduction import ForkingPickler
from types import FunctionType
import cloudpickle

assert sys.version_info >= (3, 8), 'python3.8 or greater required to use reducer_override'

def reducer_override(obj):
    if type(obj) is FunctionType:
        return (cloudpickle.loads, (cloudpickle.dumps(obj),))
    else:
        return NotImplemented

# Monkeypatch our function reducer into the pickler for multiprocessing.
# Without this line, the main block will not work on windows or macOS.
# Alterntively, moving the defintionn of foo outside of the if statement
# would make the main block work on windows or macOS (when run from
# the command line).
ForkingPickler.reducer_override = staticmethod(reducer_override)

In [7]:
chunksize = 5000

chunks = [list(arda_tokenizer.vocab.keys())[i:i + chunksize] for i in range(0, len(arda_tokenizer.vocab), chunksize)]
len(chunks), chunks[0][:5]

(52, ['fatalizm', 'anlatımına', 'Tör', 'Sülfür', 'Pediatri'])

In [9]:
counter = 0
if __name__ == '__main__':
  def find_tokens_map(tr_tokens):
    global counter
    token_maps = []
    for tr_token in tr_tokens:
        tr_token_id = arda_tokenizer.vocab[tr_token]
        hf_token_ids = hf_tokenizer.encode(tr_token)

        token_maps.append({
          "tr_token": tr_token,
          "tr_token_id": tr_token_id,
          "hf_token_ids": hf_token_ids
          })
        counter += 1
        print(counter, tr_token, tr_token_id, len(hf_token_ids))
    return token_maps
  
  with Pool(100) as p:
    token_maps_list = p.map(find_tokens_map, chunks)
    

4942 muhayyel 210263 5
4961 Vahdet 71320 4
4890 vaadiyle 122138 4
4888 arazileri 65453 4
4893 Topikal 202033 3
4872 İncil 41947 4
4940 çoğulla 224272 4
4947 Dokunca 217238 3
4975 Coupon 126777 2
4987 ugurlu 188434 4
4965 arkadaşa 62164 4
4934 QLED 150265 3
4989 atepe 42332 3
4941 sıcakkanlı 106541 5
4979 müşrikler 116459 4
4954 Miu 199166 3
4892 eira 63945 2
4976 Hayek 117441 3
4943 Semester 250206 2
4962 aktarırlar 195963 4
4932 Ekstraklasa 153808 5
4943 Güleç 86591 4
4962 1669 115738 5
4831 Proceedings 101165 2
4889 executivesr 195526 4
4891 Sophie 63356 2
4942 tima 80768 2
4873 parmakları 80075 4
4894 ɽ 589 2
4976 lefkosabet 153359 4
4948 cılığın 46667 4
4988 Ⱑ 5266 4
4943 KWD 165211 3
4966 panosu 26761 3
4935 java 79307 2
4980 ;" 35955 2
4893 Lyon 44337 2
4990 direktörler 41684 4
4977 LAYACAK 4955 görünü 23859 3
4944 cebel 131146 3
154680 4
4890 mü 16227 2
4944 Represent 136347 2
4963 ▁spille 236979 2
4933 ⮚ 5228 2
4944 boynun 97142 4
4963 Ekspresyonizm 211711 6
4874 Bulunamadı 888

In [10]:
import pandas as pd
token_list_for_df = []
for token_maps in token_maps_list:
  for token_map in token_maps:
    token_list_for_df.append(token_map)

df = pd.DataFrame(token_list_for_df)
df

,tr_token,tr_token_id,hf_token_ids
0,fatalizm,222465,"[2, 70021, 89314]"
1,anlatımına,158666,"[2, 481, 2402, 8994, 23951]"
2,Tör,30651,"[2, 235279, 2864]"
3,Sülfür,207213,"[2, 235277, 9651, 19759]"
4,Pediatri,93650,"[2, 235295, 10707, 22370]"
...,...,...,...
255995,demokrasi,28925,"[2, 14970, 8936, 542]"
255996,Bolt,104994,"[2, 112015]"
255997,nicel,102095,"[2, 5871, 521]"
255998,dığın,27935,"[2, 235258, 188791]"


In [11]:
# remove first token from llama_token_ids and gemma2_token_ids
df["hf_token_ids"] = df["hf_token_ids"].apply(lambda x: x[1:])
df

,tr_token,tr_token_id,hf_token_ids
0,fatalizm,222465,"[70021, 89314]"
1,anlatımına,158666,"[481, 2402, 8994, 23951]"
2,Tör,30651,"[235279, 2864]"
3,Sülfür,207213,"[235277, 9651, 19759]"
4,Pediatri,93650,"[235295, 10707, 22370]"
...,...,...,...
255995,demokrasi,28925,"[14970, 8936, 542]"
255996,Bolt,104994,[112015]
255997,nicel,102095,"[5871, 521]"
255998,dığın,27935,"[235258, 188791]"


In [12]:
hf_model.lm_head.weight.requires_grad

True

In [70]:
import torch

gemma_embeddings = hf_model.lm_head.weight
embeddings = torch.zeros(len(df), gemma_embeddings.shape[1])# torch.nn.parameter.Parameter(gemma_embeddings[:len(df)]) 
# torch.zeros(len(df), gemma_embeddings.shape[1])
type(embeddings), type(gemma_embeddings)

(torch.Tensor, torch.nn.parameter.Parameter)

In [57]:
# change first token embeddings
# embeddings[0] = gemma_embeddings[0]
# RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.


In [71]:
""" for token_map in token_list:
    index = token_map['tr_token_id']
    cosmos_token_ids = token_map['cosmos_token_ids']
    embedding = cosmos_embeddings[cosmos_token_ids[0]]
    sum_embedding = embedding
    for cosmos_token_id in cosmos_token_ids[1:]:
        embedding = embedding + cosmos_embeddings[cosmos_token_id]
    if len(cosmos_token_ids) > 1:
        embedding = embedding / len(cosmos_token_ids)        
    embeddings[index] = embedding

embeddings[0] """

for i, row in df.iterrows():
    tr_token_id = row["tr_token_id"]
    hf_token_ids = row["hf_token_ids"]
    embedding = gemma_embeddings[hf_token_ids[0]]
    sum_embedding = embedding
    for hf_token_id in hf_token_ids[1:]:
        embedding = embedding + gemma_embeddings[hf_token_id]
    if len(hf_token_ids) > 1:
        embedding = embedding / len(hf_token_ids)
    embeddings[i] = embedding



In [83]:
embeddings[158666][120:200]

tensor([ 6.8461e-03, -1.4054e-02,  3.2641e-03, -7.5798e-03,  9.6873e-04,
         1.0500e-02,  8.9188e-03,  3.7047e-03, -1.0767e-02, -1.1299e-02,
         1.4242e-04,  3.0322e-03, -3.9815e-02,  1.3648e-02,  2.7771e-03,
        -2.2316e-03, -2.2634e-03, -4.8012e-02,  1.1195e-02,  2.1362e-04,
        -4.6041e-02,  1.3148e-03,  1.5854e-02, -3.2043e-04, -1.7802e-04,
        -3.2743e-03,  7.9587e-03, -4.7836e-03,  2.2995e-02,  1.6853e-02,
         1.3034e-03,  1.6477e-02,  2.3832e-02,  6.4155e-03,  9.9748e-03,
         1.2299e-03,  6.8398e-03, -2.9498e-02, -2.4348e-02, -1.9042e-03,
        -3.6621e-04,  8.8965e-03,  1.0783e-03, -2.0157e-02, -2.0807e-02,
        -9.3444e-03, -2.9297e-03,  2.8966e-03,  4.7493e-04,  1.1304e-02,
        -7.1446e-03,  5.5491e-03,  1.5828e-02, -1.5137e-02,  2.3407e-02,
        -1.5961e-02, -1.6998e-02,  1.5761e-03, -3.3786e-03,  4.3569e-02,
         1.9188e-02,  4.3439e-02, -1.4908e-02, -3.5400e-03,  6.4977e-03,
        -7.6294e-05, -1.8097e-02,  3.7072e-03,  3.0

In [73]:
p_embedding = torch.nn.Parameter(embeddings)
type(p_embedding), p_embedding.requires_grad

(torch.nn.parameter.Parameter, True)

In [74]:
# change the embedding layer
hf_model.lm_head.weight = p_embedding
hf_model.lm_head.weight.shape

torch.Size([256000, 2304])

In [25]:
hf_model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps

In [82]:
tr_tokenizer.tokenize(example_text)

['Nasıl',
 'sın',
 ',',
 'iyi',
 'mis',
 'in',
 '?',
 'Tür',
 'k',
 'çe',
 'cevap',
 'ver',
 'lütfen',
 '!']

In [78]:
example_text = "Nasılsın, iyi misin? Türkçe cevap ver lütfen!"
input_ids = arda_tokenizer.encode(example_text, return_tensors="pt")
u_input_ids = umut_tokenizer.encode(example_text, return_tensors="pt")
input_ids, u_input_ids

(tensor([[17408, 16735,    18, 16291, 17675, 15838,    37, 19473,    81, 16346,
          17578, 15939, 18824,     7]]),
 tensor([[58058, 55623,  1659, 54300, 55716, 53909,  1668, 55554, 76194, 54134,
          56312, 53996, 67741,  1648]]))

In [75]:
# after changing of the embedding layer
output_token_ids = hf_model.generate(input_ids, max_length=100)
output_token_ids, arda_tokenizer.decode(output_token_ids[0])

(tensor([[ 17408,  16735,     18,  16291,  17675,  15838,     37,  19473,     81,
           16346,  17578,  15939,  18824,      7, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627, 120627,
          120627]]),
 'Nasıl sın , iyi mis in ? Tür k çe cevap ver lütfen ! 1738 1738 1738 1738 17